# MySQL 연동 통합 실습

이 노트북은 Python과 MySQL을 연동하여 데이터베이스 작업을 수행하는 방법을 단계별로 학습하는 자료입니다.

## 목차
1. [환경 설정 및 패키지 설치](#1.-환경-설정-및-패키지-설치)
2. [MySQL 계정 생성 및 권한 부여](#2.-MySQL-계정-생성-및-권한-부여)
3. [기본 연결 및 데이터 조회](#3.-기본-연결-및-데이터-조회)
4. [다양한 데이터 가져오기 방법](#4.-다양한-데이터-가져오기-방법)
5. [Dictionary 커서 사용](#5.-Dictionary-커서-사용)
6. [데이터 삽입(INSERT) 실습](#6.-데이터-삽입(INSERT)-실습)
7. [완전한 CRUD 시스템](#7.-완전한-CRUD-시스템)
8. [성적 관리 시스템 예제](#8.-성적-관리-시스템-예제)


## 1. 환경 설정 및 패키지 설치

Python에서 MySQL을 사용하기 위해 필요한 패키지를 설치합니다.


In [ ]:
# 필요한 패키지 설치 (터미널에서 실행)
# conda activate myenv1 
# pip install pymysql 
# MySQL 8부터 문제 있을 경우: pip install cryptography 

# 패키지 import
import pymysql
print("PyMySQL 패키지가 성공적으로 import되었습니다!")


## 2. MySQL 계정 생성 및 권한 부여

Python에서 MySQL에 접속하기 위해 전용 계정을 생성하고 권한을 부여합니다.

### MySQL 관리자(root) 계정으로 접속하여 다음 명령어를 실행하세요:


In [ ]:
# MySQL에서 실행할 계정 생성 및 권한 부여 명령어
sql_commands = """
-- 계정 생성 (localhost에서만 접근 가능)
-- localhost = 루프백주소(127.0.0.1)
-- 'user02'@'localhost'로 로컬서버만 접근가능한 계정입니다

CREATE USER 'user02'@'localhost' IDENTIFIED BY 'qwer1234';

-- 권한 부여 (mydb 데이터베이스의 모든 테이블에 대한 모든 권한)
GRANT ALL PRIVILEGES ON mydb.* TO 'user02'@'localhost';

-- 권한 새로고침
FLUSH PRIVILEGES;
"""

print("MySQL에서 다음 명령어를 실행하세요:")
print(sql_commands)


## 3. 기본 연결 및 데이터 조회

Python에서 MySQL에 연결하고 기본적인 데이터 조회를 수행합니다.

### 기본 연결 과정:
1. 데이터베이스 연결
2. 커서 생성
3. 쿼리 실행
4. 결과 조회
5. 연결 종료


In [ ]:
# MySQL 연결 설정
conn = pymysql.connect(
    host='localhost',      # IP 주소 (localhost = 127.0.0.1)
    user='user02',         # 사용자 ID
    password='qwer1234',   # 비밀번호
    db='mydb',            # 데이터베이스명
    port=3306             # 포트번호 (기본값: 3306)
)

# 커서 생성 (기본 tuple 타입)
cursor = conn.cursor()
print("MySQL 접속 성공!")

# 연결 정보 확인
print(f"연결된 데이터베이스: {conn.db}")
print("PyMySQL을 통한 MySQL 연결이 완료되었습니다.")


## 4. 다양한 데이터 가져오기 방법

PyMySQL에서 제공하는 다양한 데이터 가져오기 방법을 학습합니다.

- `fetchall()`: 모든 데이터 가져오기
- `fetchone()`: 한 건의 데이터 가져오기  
- `fetchmany(n)`: 지정한 개수만큼 가져오기


In [ ]:
# 1. fetchall() - 모든 데이터 가져오기
print("=== 1. fetchall() - 모든 데이터 가져오기 ===")
sql = "SELECT * FROM emp"
cursor.execute(sql)
rows = cursor.fetchall()  # 데이터를 tuple 타입으로 가져온다

print(f"총 {len(rows)}건의 데이터가 조회되었습니다.")
for row in rows:
    print(f"데이터 타입: {type(row)}, 내용: {row}")
    if len(rows) > 5:  # 데이터가 많을 경우 5개만 출력
        break


In [ ]:
# 2. fetchone() - 한 건의 데이터 가져오기
print("\n=== 2. fetchone() - 한 건의 데이터 가져오기 ===")
sql = "SELECT * FROM emp WHERE empno = 7369"
cursor.execute(sql)
row = cursor.fetchone()

if row:
    print(f"조회된 데이터: {row}")
else:
    print("데이터가 없습니다.")


In [ ]:
# 3. fetchmany(n) - 지정한 개수만큼 가져오기
print("\n=== 3. fetchmany(3) - 3개만 가져오기 ===")
sql = "SELECT * FROM emp WHERE empno < 8000"
cursor.execute(sql)
rows = cursor.fetchmany(3)  # 앞에서 3개만

print(f"조회된 데이터 {len(rows)}건:")
for i, row in enumerate(rows, 1):
    print(f"{i}. {row}")


## 5. Dictionary 커서 사용

기본 커서는 데이터를 tuple 타입으로 반환하여 인덱싱과 슬라이싱만 지원합니다.  
Dictionary 커서를 사용하면 컬럼명으로 데이터에 접근할 수 있어 더 편리합니다.

**장점:**
- `row["컬럼명"]` 형태로 직관적인 접근 가능
- 코드 가독성 향상
- 컬럼 순서 변경에 영향받지 않음


In [ ]:
# Dictionary 커서 생성 및 사용
print("=== Dictionary 커서 사용 예제 ===")

# Dictionary 커서를 위한 import
from pymysql.cursors import DictCursor

# Dictionary 커서로 변경
cursor = conn.cursor(DictCursor)

# 특정 사원 정보 조회
ename = "SCOTT"
sql = "SELECT empno, ename, sal FROM emp WHERE ename = %s"
print(f"실행할 SQL: {sql}")
print(f"검색할 이름: {ename}")

cursor.execute(sql, (ename,))  # 매개변수화 쿼리 사용 (SQL 인젝션 방지)
rows = cursor.fetchall()

print(f"\n조회된 데이터 개수: {len(rows)}")
for row in rows:
    # Dictionary 형태로 접근 가능
    print(f"사원번호: {row.get('empno')}, 이름: {row.get('ename')}, 급여: {row.get('sal')}")


## 6. 데이터 삽입(INSERT) 실습

데이터베이스에 새로운 데이터를 삽입하는 방법을 학습합니다.

### 중요사항:
- INSERT, UPDATE, DELETE 작업 후에는 반드시 `commit()`을 호출해야 합니다.
- 매개변수화 쿼리를 사용하여 SQL 인젝션을 방지합니다.
- `%s` 플레이스홀더를 사용합니다. (MySQL에서는 `?` 대신 `%s` 사용)


In [ ]:
# 1. 다음 사원번호 조회 (max 함수 활용)
print("=== 데이터 삽입 실습 ===")

# max 함수가 데이터가 한 건도 없을 때 null을 반환하므로 ifnull 사용
sql = "SELECT IFNULL(MAX(empno), 0) + 1 AS next_id FROM emp"
cursor.execute(sql)
row = cursor.fetchone()

if row:
    next_empno = row.get('next_id', 1)  # 기본값으로 1 설정
    print(f"다음 사원번호: {next_empno}")
    
    # 2. 새로운 사원 데이터 삽입
    sql = """
        INSERT INTO emp(empno, ename, sal)
        VALUES(%s, %s, %s) 
    """
    
    # 매개변수화 쿼리로 안전하게 데이터 삽입
    cursor.execute(sql, (next_empno, '김철수', 5000))
    
    # 반드시 commit을 해야 실제 데이터베이스에 반영됩니다!
    conn.commit()
    
    print(f"새로운 사원이 추가되었습니다: 사원번호 {next_empno}, 이름: 김철수, 급여: 5000")
else:
    print("사원번호 조회에 실패했습니다.")


## 7. 완전한 CRUD 시스템

CRUD(Create, Read, Update, Delete) 기능을 모두 포함한 완전한 데이터 관리 시스템을 구현합니다.

### CRUD 기능:
- **C**reate: 데이터 생성 (INSERT)
- **R**ead: 데이터 조회 (SELECT)  
- **U**pdate: 데이터 수정 (UPDATE)
- **D**elete: 데이터 삭제 (DELETE)


In [ ]:
# CRUD 함수들 정의

def insert_employee():
    """새로운 사원 추가"""
    print("\n=== 새 사원 추가 ===")
    name = input("이름: ")
    job = input("직무: ")
    sal = input("급여: ")
    
    # 다음 사원번호 생성
    sql = "SELECT IFNULL(MAX(empno), 0) + 1 AS next_id FROM emp"
    cursor.execute(sql)
    row = cursor.fetchone()
    next_empno = row.get('next_id', 1) if row else 1
    
    # 데이터 삽입
    sql = """
        INSERT INTO emp(empno, ename, job, sal, hiredate)
        VALUES (%s, %s, %s, %s, NOW())
    """
    cursor.execute(sql, (next_empno, name, job, sal))
    conn.commit()
    
    print(f"사원이 추가되었습니다. (사원번호: {next_empno})")

def read_employees():
    """사원 목록 조회"""
    print("\n=== 사원 목록 ===")
    sql = """
        SELECT empno, ename, job, sal, hiredate
        FROM emp
        ORDER BY empno
    """
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    if rows:
        print(f"{'사원번호':<8} {'이름':<10} {'직무':<10} {'급여':<8} {'입사일'}")
        print("-" * 50)
        for row in rows:
            print(f"{row.get('empno', ''):<8} {row.get('ename', ''):<10} {row.get('job', ''):<10} {row.get('sal', ''):<8} {row.get('hiredate', '')}")
    else:
        print("등록된 사원이 없습니다.")

def update_employee():
    """사원 정보 수정"""
    print("\n=== 사원 정보 수정 ===")
    empno = input("수정할 사원번호: ")
    name = input("새 이름: ")
    job = input("새 직무: ")
    sal = input("새 급여: ")
    
    sql = """
        UPDATE emp
        SET ename = %s, job = %s, sal = %s
        WHERE empno = %s
    """
    result = cursor.execute(sql, (name, job, sal, empno))
    conn.commit()
    
    if cursor.rowcount > 0:
        print(f"사원번호 {empno}의 정보가 수정되었습니다.")
    else:
        print("해당 사원을 찾을 수 없습니다.")

def delete_employee():
    """사원 삭제"""
    print("\n=== 사원 삭제 ===")
    empno = input("삭제할 사원번호: ")
    
    # 삭제 전 확인
    sql = "SELECT ename FROM emp WHERE empno = %s"
    cursor.execute(sql, (empno,))
    row = cursor.fetchone()
    
    if row:
        confirm = input(f"'{row.get('ename')}'님을 정말 삭제하시겠습니까? (y/N): ")
        if confirm.lower() == 'y':
            sql = "DELETE FROM emp WHERE empno = %s"
            cursor.execute(sql, (empno,))
            conn.commit()
            print("사원이 삭제되었습니다.")
        else:
            print("삭제가 취소되었습니다.")
    else:
        print("해당 사원을 찾을 수 없습니다.")

print("CRUD 함수들이 정의되었습니다.")


## 8. 성적 관리 시스템 예제

학생의 성적을 관리하는 완전한 시스템 예제입니다. (원본 mysql연동3.py 기반)

### 기능:
1. 성적 목록 조회
2. 새 성적 추가
3. 성적 수정
4. 성적 삭제

### 주의사항:
- `tb_score` 테이블이 미리 생성되어 있어야 합니다.
- 실제 실행하려면 테이블 구조에 맞게 코드를 수정해야 할 수 있습니다.


In [ ]:
# 성적 관리 시스템 함수들

def insert_score():
    """새 성적 추가"""
    print("\n=== 성적 추가 ===")
    name = input("이름: ")
    kor = input("국어: ")
    eng = input("영어: ")
    mat = input("수학: ")
    
    sql = """
        INSERT INTO tb_score(name, kor, eng, mat, regdate)
        VALUES (%s, %s, %s, %s, NOW())
    """
    try:
        cursor.execute(sql, (name, kor, eng, mat))
        conn.commit()  # 반드시 해야 한다
        print("성적이 추가되었습니다.")
    except Exception as e:
        print(f"성적 추가 중 오류 발생: {e}")

def update_score():
    """성적 수정"""
    print("\n=== 성적 수정 ===")
    score_id = input("수정할 아이디: ")
    name = input("이름: ")
    kor = input("국어: ")
    eng = input("영어: ")
    mat = input("수학: ")
    
    sql = """
        UPDATE tb_score
        SET name = %s, kor = %s, eng = %s, mat = %s
        WHERE id = %s
    """
    try:
        cursor.execute(sql, (name, kor, eng, mat, score_id))
        conn.commit()
        if cursor.rowcount > 0:
            print("성적이 수정되었습니다.")
        else:
            print("해당 ID를 찾을 수 없습니다.")
    except Exception as e:
        print(f"성적 수정 중 오류 발생: {e}")

def delete_score():
    """성적 삭제"""
    print("\n=== 성적 삭제 ===")
    score_id = input("삭제할 아이디: ")
    
    sql = "DELETE FROM tb_score WHERE id = %s"
    try:
        cursor.execute(sql, (score_id,))
        conn.commit()
        if cursor.rowcount > 0:
            print("성적이 삭제되었습니다.")
        else:
            print("해당 ID를 찾을 수 없습니다.")
    except Exception as e:
        print(f"성적 삭제 중 오류 발생: {e}")

def show_scores():
    """성적 목록 조회"""
    print("\n=== 성적 목록 ===")
    sql = """
        SELECT id, name, kor, eng, mat,
               (kor + eng + mat) AS total,
               (kor + eng + mat) / 3 AS avg,
               regdate
        FROM tb_score
        ORDER BY id
    """
    try:
        cursor.execute(sql)
        rows = cursor.fetchall()
        
        if rows:
            print(f"{'ID':<4} {'이름':<8} {'국어':<4} {'영어':<4} {'수학':<4} {'총점':<4} {'평균':<6} {'등록일'}")
            print("-" * 60)
            for row in rows:
                avg_score = f"{row.get('avg', 0):.1f}" if row.get('avg') else "0.0"
                print(f"{row.get('id', ''):<4} {row.get('name', ''):<8} {row.get('kor', ''):<4} {row.get('eng', ''):<4} {row.get('mat', ''):<4} {row.get('total', ''):<4} {avg_score:<6} {row.get('regdate', '')}")
        else:
            print("등록된 성적이 없습니다.")
    except Exception as e:
        print(f"성적 조회 중 오류 발생: {e}")

def score_management_menu():
    """성적 관리 메뉴 시스템"""
    print("\\n=== 성적 관리 시스템 ===")
    print("주의: 실제 실행하려면 tb_score 테이블이 필요합니다.")
    print("테이블 구조: id(AUTO_INCREMENT), name(VARCHAR), kor(INT), eng(INT), mat(INT), regdate(DATETIME)")
    
    # 실제 메뉴는 주석으로 제공 (테이블이 없을 수 있으므로)
    menu_code = '''
    while True:
        sel = input("1.목록 2.추가 3.수정 4.삭제 0.종료: ")
        if sel == "1":
            show_scores()
        elif sel == "2":
            insert_score()
        elif sel == "3":
            update_score()
        elif sel == "4":
            delete_score()
        elif sel == "0":
            break
        else:
            print("잘못된 선택입니다.")
    '''
    print("메뉴 실행 코드:")
    print(menu_code)

print("성적 관리 시스템 함수들이 정의되었습니다.")


## 9. 연결 종료 및 정리

데이터베이스 작업이 완료되면 반드시 연결을 종료해야 합니다.


In [ ]:
# 데이터베이스 연결 종료
def close_connection():
    """데이터베이스 연결 종료"""
    if conn:
        conn.close()
        print("MySQL 연결이 종료되었습니다.")

# 실제 연결 종료 (필요시 실행)
# close_connection()

print("연결 종료 함수가 정의되었습니다.")
print("실제 종료하려면 close_connection()을 호출하세요.")


## 10. 요약 및 주요 포인트

이 노트북에서 학습한 내용을 정리합니다.

### 주요 학습 내용:

1. **환경 설정**: PyMySQL 패키지 설치 및 import
2. **계정 관리**: MySQL 사용자 계정 생성 및 권한 부여
3. **연결 관리**: 데이터베이스 연결, 커서 생성, 연결 종료
4. **데이터 조회**: fetchall(), fetchone(), fetchmany() 활용
5. **Dictionary 커서**: 컬럼명으로 데이터 접근
6. **CRUD 작업**: Create, Read, Update, Delete 구현
7. **트랜잭션**: commit() 사용법
8. **보안**: 매개변수화 쿼리로 SQL 인젝션 방지

### 핵심 원칙:

#### 1. 연결 관리
```python
# 연결
conn = pymysql.connect(host, user, password, db, port)
cursor = conn.cursor(DictCursor)

# 작업 수행
cursor.execute(sql, params)

# 연결 종료
conn.close()
```

#### 2. 트랜잭션 관리
- **SELECT**: commit 불필요
- **INSERT/UPDATE/DELETE**: 반드시 `conn.commit()` 호출

#### 3. 보안 (SQL 인젝션 방지)
```python
# 나쁜 예 (SQL 인젝션 위험)
sql = f"SELECT * FROM emp WHERE name = '{name}'"

# 좋은 예 (매개변수화 쿼리)
sql = "SELECT * FROM emp WHERE name = %s"
cursor.execute(sql, (name,))
```

#### 4. 에러 처리
```python
try:
    cursor.execute(sql, params)
    conn.commit()
except Exception as e:
    print(f"오류 발생: {e}")
    conn.rollback()  # 필요시 롤백
```

### 추가 학습 권장사항:

1. **커넥션 풀링**: 대량 접속 시 성능 향상
2. **ORM 도구**: SQLAlchemy, Django ORM 등
3. **비동기 처리**: aiomysql 등 비동기 라이브러리
4. **데이터베이스 설계**: 정규화, 인덱스 설계
5. **성능 최적화**: 쿼리 최적화, 인덱스 활용

이제 Python과 MySQL을 활용한 데이터베이스 프로그래밍의 기초를 마스터했습니다!
